In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from utils.help_func import create_path
class SegmentationNpyDataset(Dataset):
    def __init__(self, images_dir, masks_dir):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_files = sorted([
            f for f in os.listdir(images_dir) if f.endswith('.npy')
        ])
        self.mask_files = sorted([
            f for f in os.listdir(masks_dir) if f.endswith('.npy')
        ])
        assert len(self.image_files) == len(self.mask_files), "Mismatch between image and mask counts"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = np.load(os.path.join(self.images_dir, self.image_files[idx]))  # [3, 224, 224]
        mask = np.load(os.path.join(self.masks_dir, self.mask_files[idx]))     # [1, 224, 224]

        image = torch.from_numpy(image).float()  # [3, H, W]
        mask = torch.from_numpy(mask).long()     # [1, H, W]
        return image, mask

# === Initialize datasets ===
train_dataset = SegmentationNpyDataset(
    "./finetune_data/segmentation/train/images",
    "./finetune_data/segmentation/train/masks"
)
test_dataset = SegmentationNpyDataset(
    "./finetune_data/segmentation/test/images",
    "./finetune_data/segmentation/test/masks"
)

# === Create DataLoaders ===
dataloader_train = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [2]:
from multi_seg.models_mae_finetune_seg import MaskedAutoencoderViTMultiSeg
from functools import partial
import torch
import torch.nn as nn
labels = ['Left/Right-Cerebral-White-Matter', 'Left/Right-Cerebral-Cortex',
'Left/Right-Cerebellum-White-Matter', 'Left/Right-Cerebellum-Cortex', 'Left/Right-Thalamus', 'Left/Right-Caudate',
'Left/Right-Putamen', 'Left/Right-Pallidum', 'Brain-Stem ', 'Left/Right-Hippocampus', 'Left/Right-Amygdala', 'CSF',
'WM-hypointensities']
mode = 'mae_unet_fuse'
INPUT_SIZE = 224
# mode = 'CNN'
mae_model = MaskedAutoencoderViTMultiSeg(
        patch_size=16, embed_dim=768, depth=12, num_heads=12,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, norm_layer=partial(nn.LayerNorm, eps=1e-6), num_classes=len(labels) + 1, mode=mode,
                 drop=0.1, attn_drop=0.1, drop_path=0.05)

pretrain_path = './saved_models/mae_vit_base_patch16_pretrain_test0.75_E30/model_E30.pt'

missing, unexpected = mae_model.load_state_dict(torch.load(pretrain_path)['model_state_dict'], strict=False)  # strict=False ignores unmatched keys
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

from multi_seg.mae_unet_fuse import UnetWithMAEFusion
from modeling.unet import Unet
CHANS = 64
unet_model = Unet(in_chans=3,out_chans=len(labels) + 1, chans=CHANS, if_classify= False,  dim=INPUT_SIZE, mlp_ratio = 32)

model = UnetWithMAEFusion(unet=unet_model, mae_model=mae_model, custom_fusion_modes=['concat', 'concat', 'concat', 'concat', 'concat', 'concat', 'concat', 'concat', 'concat'], mae_indices = [0, 2, 5, 8, 11, 8, 5, 2, 0], skip_connect = True, skip_type = 'concat')

Missing keys: []
Unexpected keys: []


In [3]:
from multi_seg.dice_loss_seg import CombinedGDLFocalCELoss, softmax_helper
from multi_seg.dice_loss_seg import CombinedDiceFocalCELoss, softmax_helper
# gdl Loss settings
gdl_kwargs = {
    'apply_nonlin': softmax_helper,   # apply softmax inside
    'batch_dice': False,              # per sample Dice
    'do_bg': True,                    # include background (class 0)
    'smooth': 1e-5,                    # smoothing factor
    'square': False,
    'square_volumes': False,
}

# Dice Loss settings
dice_kwargs = {
    'apply_nonlin': softmax_helper,  # Apply softmax inside DiceLoss
    'batch_dice': False,              # Per-sample dice (not batch-level)
    'do_bg': True,                    # Include background class
    'smooth': 1e-5                    # Smoothing factor
}

# Focal Loss settings
focal_kwargs = {
    'gamma': 2.0,
    'alpha': 0.25,
    'reduction': 'mean',
    'apply_nonlin': softmax_helper    # Apply softmax inside FocalLoss too
}

# CrossEntropy Loss settings
ce_kwargs = {
    'reduction': 'mean'
}

# (Optional) Weights for each sub-loss (dice, focal, ce)
weights = (1.0, 1.0, 1.0)


loss_fn = CombinedDiceFocalCELoss(
    dice_kwargs=dice_kwargs,
    focal_kwargs=focal_kwargs,
    ce_kwargs=ce_kwargs,
    weights=weights
)

# loss_fn = nn.CrossEntropyLoss(
#     reduction='mean'
# )

import torch.optim as optim

# ---- Optimizer ----
lr = 1e-4
weight_decay= 1e-2
optimizer = optim.AdamW(
    model.parameters(),
    lr=lr,
    weight_decay=weight_decay
)


In [4]:
from multi_seg.train_mae_finetune_seg import Trainer
freeze_mae_encoder = True
freeze_mae_encoder_decoder = True
force_float32 = False
TRAIN_EPOCHS = 200
freeze_start_epoch = None
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if freeze_mae_encoder:
    if freeze_mae_encoder_decoder:
        path_save = "../saved_models/segment/freeze_ED" + str(freeze_start_epoch) +"_mae_unet" + str(CHANS) + "concat_" + mode + "_e" + str(TRAIN_EPOCHS) + '/'
    else:#concat_fuze_
        path_save = "../saved_models/segment/freeze_E" + str(freeze_start_epoch) +"_mae_unet" + str(CHANS) + "concat_" + mode + "_e" + str(TRAIN_EPOCHS) + '/'
else: # use
    path_save = "../saved_models/segment/unfreeze" + str(freeze_start_epoch) +"_mae_unet" + str(CHANS) + "concat_" + mode + "_e" + str(TRAIN_EPOCHS) + '/'
create_path(path_save)
trainer = Trainer(
    loader_train=dataloader_train,      # your training DataLoader
    loader_test=dataloader_test,        # your test/validation DataLoader
    my_model=model,                 # your MaskedAutoencoderViTMultiSeg model
    my_loss=loss_fn,
    optimizer=optimizer,
    RESUME_EPOCH=0,                 # set >0 if you load checkpoints
    PATH_MODEL=path_save,    # where you want to save checkpoints
    device=device,                  # or 'cuda:0', depending on your setup
    task='segment',                 # since you are doing segmentation
    freeze_mae_encoder=freeze_mae_encoder,       # if you want to freeze encoder
    freeze_mae_encoder_decoder=freeze_mae_encoder_decoder,  # if you want to freeze encoder+decoder
    force_float32 = force_float32,
    unfreeze_at_epoch = freeze_start_epoch,
    weight_decay=weight_decay,
    num_classes= len(labels) + 1,
)


Path already exists.


D:\Mengyu_Li\General_Dataloader_Git_V1\multi_seg\train_mae_finetune_seg.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
D:\Mengyu_Li\General_Dataloader_Git_V1\multi_seg\train_mae_finetune_seg.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()


In [ ]:
model = trainer.train(epochs=TRAIN_EPOCHS, show_step=200, show_test=True)

MAE freeze mode is enabled: True
Total params: 150088974
Trainable params: 38181134


Training:   0%|          | 0/200 [00:00<?, ?it/s]

images_input is a  <class 'torch.Tensor'> with shape torch.Size([32, 3, 224, 224]) max:  tensor(1., device='cuda:0') min:  tensor(0., device='cuda:0')
images type: torch.float32
labels_input is a  <class 'torch.Tensor'> with shape torch.Size([32, 224, 224]) max:  tensor(13, device='cuda:0') min:  tensor(0, device='cuda:0')
labels type: torch.int64
logits is a  <class 'torch.Tensor'> with shape torch.Size([32, 14, 224, 224]) max:  tensor(1.8111, device='cuda:0', grad_fn=<MaxBackward1>) min:  tensor(-1.6269, device='cuda:0', grad_fn=<MinBackward1>)
logits type: torch.float32
⏱️ Batch 0 time: 1.2631 seconds
⏱️ Batch 1 time: 0.5220 seconds
⏱️ Batch 2 time: 0.5150 seconds
⏱️ Batch 3 time: 0.5336 seconds
⏱️ Batch 4 time: 0.5250 seconds
⏱️ Batch 5 time: 0.5120 seconds
⏱️ Batch 6 time: 0.5230 seconds
⏱️ Batch 7 time: 0.5082 seconds
⏱️ Batch 8 time: 0.5095 seconds
⏱️ Batch 9 time: 0.5180 seconds
⏱️ Batch 10 time: 0.5160 seconds
⏱️ Batch 11 time: 0.5190 seconds
⏱️ Batch 12 time: 0.5145 seconds
⏱